# Data Collection and Target Setting

In [2]:
import pandas as pd
import requests
import json
import time
import numpy as np

pd.set_option('display.max_columns', None)


In [21]:
#Filtering to games completed as of mid-way through August 23rd
game_list = pd.read_csv('../raw_data/games.csv', index_col="id")
game_list = game_list[game_list["status"] == "closed"]
g_list = game_list.index

In [22]:
# setting naming convention for the columns that the json files are read into the Dataframe
columns = ['id', 'game_id', 'inning', 'side', 'hitter_id', 'hitter_hand', 'pitcher_id', 'pitcher_hand', 'description', 'temp_f', 'weather_condition', 'humidity', 'wind_speed_mph']

In [23]:
#Create empty DataFrame to concat to in the next code cell
at_bat_df = pd.DataFrame(columns=columns)

In [25]:
#Parsing through JSON to pull the key data into a dataframe

columns = ['id', 'game_id', 'inning', 'side', 'hitter_id', 'hitter_hand', 'pitcher_id', 'pitcher_hand', 'description', 'temp_f', 'weather_condition', 'humidity', 'wind_speed_mph']

for g in g_list:
    with open(f'../data/plate_app_data/{g}.json') as user_file:
        api_c = user_file.read()
    api_c = json.loads(api_c)
    
    for ing in range(1, len(api_c["game"]["innings"])):
        for hlf in range(2):
            for atb in range(len(api_c["game"]["innings"][ing]["halfs"][hlf]["events"])):
                
                if 'at_bat' not in api_c["game"]["innings"][ing]["halfs"][hlf]["events"][atb]:
                    continue
                
                if 'pitcher_id' not in api_c["game"]["innings"][ing]["halfs"][hlf]["events"][atb]["at_bat"]:
                    continue
                
                id = api_c["game"]["innings"][ing]["halfs"][hlf]["events"][atb]["at_bat"]["id"]
                game_id = api_c["game"]["id"]
                inning = api_c["game"]["innings"][ing]["number"]
                side = api_c["game"]["innings"][ing]["halfs"][hlf]["half"]

                hitter_id = api_c["game"]["innings"][ing]["halfs"][hlf]["events"][atb]["at_bat"]["hitter_id"]
                hitter_hand = api_c["game"]["innings"][ing]["halfs"][hlf]["events"][atb]["at_bat"]["hitter_hand"]
                
                pitcher_id = api_c["game"]["innings"][ing]["halfs"][hlf]["events"][atb]["at_bat"]["pitcher_id"]
                pitcher_hand = api_c["game"]["innings"][ing]["halfs"][hlf]["events"][atb]["at_bat"]["pitcher_hand"]
                
                if 'description' not in api_c["game"]["innings"][ing]["halfs"][hlf]["events"][atb]["at_bat"]:
                    continue
                
                description = api_c["game"]["innings"][ing]["halfs"][hlf]["events"][atb]["at_bat"]["description"]

                if 'weather' in api_c["game"]["innings"][ing]["halfs"][hlf]:
                    if "temp_f" not in api_c["game"]["innings"][ing]["halfs"][hlf]["weather"]["current_conditions"]:
                        temp_f = np.nan
                    else:
                        temp_f = api_c["game"]["innings"][ing]["halfs"][hlf]["weather"]["current_conditions"]["temp_f"]
                    
                    if "condition" not in api_c["game"]["innings"][ing]["halfs"][hlf]["weather"]["current_conditions"]:
                        weather_condition = np.nan
                    else:    
                        weather_condition = api_c["game"]["innings"][ing]["halfs"][hlf]["weather"]["current_conditions"]["condition"]
                    
                    if "humidity" not in api_c["game"]["innings"][ing]["halfs"][hlf]["weather"]["current_conditions"]:
                        humidity  = np.nan
                    else:
                        humidity = api_c["game"]["innings"][ing]["halfs"][hlf]["weather"]["current_conditions"]["humidity"]
                        
                    if "wind" not in api_c["game"]["innings"][ing]["halfs"][hlf]["weather"]["current_conditions"]:
                        wind_speed_mph  = np.nan
                        
                    else:  
                        wind_speed_mph = api_c["game"]["innings"][ing]["halfs"][hlf]["weather"]["current_conditions"]["wind"]["speed_mph"]
                    
                else:  
                    temp_f = np.nan
                    weather_condition = np.nan
                    humidity = np.nan
                    wind_speed_mph = np.nan
            
                _temp = pd.DataFrame(list((id, game_id, inning, side, hitter_id, hitter_hand, pitcher_id, pitcher_hand, description, temp_f, weather_condition, humidity, wind_speed_mph))).T
                
                _temp.columns = columns
                
                at_bat_df = pd.concat((at_bat_df, _temp))
                
at_bat_df = at_bat_df.set_index("id")

In [5]:
#reading .csv verison
at_bat_df = pd.read_csv("../raw_data/all_ab_raw_data.csv")
at_bat_df

,id,game_id,inning,side,hitter_id,hitter_hand,pitcher_id,pitcher_hand,description,temp_f,weather_condition,humidity,wind_speed_mph
0,e7acb70b-affd-4fe8-9dcb-8a71da23612b,00846785-f968-4867-8896-cb4d5f7a3e63,1,T,6ea2efc3-345a-4e30-8b95-744a60a3e7a5,R,738eb5e8-2ab8-4428-928a-81a31b7228de,R,Nico Hoerner flies out to left field to Luis R...,70.0,Partly cloudy,53.0,4.0
1,0bbf479e-52f5-4bd6-a0f0-445e172ea7fc,00846785-f968-4867-8896-cb4d5f7a3e63,1,T,5773d9a2-dfc1-458a-acac-67528d6618b3,R,738eb5e8-2ab8-4428-928a-81a31b7228de,R,Dansby Swanson pops out to Jared Walsh.,70.0,Partly cloudy,53.0,4.0
2,56a4f487-1e79-4383-bea2-0327559a6423,00846785-f968-4867-8896-cb4d5f7a3e63,1,T,d8968ebd-8227-44b6-b442-77d088c9f98a,L,738eb5e8-2ab8-4428-928a-81a31b7228de,R,Ian Happ walks.,70.0,Partly cloudy,53.0,4.0
3,cd1bd683-83d1-421c-a68f-a2b4cf370ba6,00846785-f968-4867-8896-cb4d5f7a3e63,1,T,aa691806-f8a0-4dc7-b833-94e59dc22a34,R,738eb5e8-2ab8-4428-928a-81a31b7228de,R,Seiya Suzuki reaches on a fielder's choice to ...,70.0,Partly cloudy,53.0,4.0
4,7b3d2210-639f-4d3a-b0d5-5e35d4fd10cd,00846785-f968-4867-8896-cb4d5f7a3e63,1,B,7af39f78-8e04-408d-a1b5-66ab3dcd99ac,L,0b4c4345-7609-4ab1-9aaa-41d56d7f1fda,R,Mickey Moniak strikes out swinging.,70.0,Partly cloudy,53.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
143083,2e72bfad-65ea-4697-8f22-1f37c756ec0c,645f60c6-1ac4-4c6b-bec8-c3b3e5c82c3f,9,B,1264322d-9ded-486b-8ef2-a2f281589db6,L,c4a8a308-1517-4c0e-a8af-f8cd824525cc,R,Wander Franco flies out to deep right field to...,77.0,Sunny,40.0,2.0
143084,3787562e-6332-424a-9952-e32b8f13a8ce,645f60c6-1ac4-4c6b-bec8-c3b3e5c82c3f,9,B,fa26a373-a427-4558-ba5e-6d6c0f0148c5,R,c4a8a308-1517-4c0e-a8af-f8cd824525cc,R,Vladimir Guerrero Jr. strikes out swinging.,77.0,Sunny,40.0,2.0
143085,1970572f-91a9-4330-b52b-06fe0d5c0741,645f60c6-1ac4-4c6b-bec8-c3b3e5c82c3f,9,B,f4a9ab1b-8458-40c9-a50f-bbef4f11fe58,L,c4a8a308-1517-4c0e-a8af-f8cd824525cc,R,Kyle Tucker walks.,77.0,Sunny,40.0,2.0
143086,2e62b9ad-8e5f-48a8-8da8-cde31500cdb8,645f60c6-1ac4-4c6b-bec8-c3b3e5c82c3f,9,B,e750ccd5-7a24-44ef-b661-0d41baa17d1c,R,c4a8a308-1517-4c0e-a8af-f8cd824525cc,R,Julio Rodríguez walks. Kyle Tucker to second.,77.0,Sunny,40.0,2.0


In [6]:
#Mapping descriptions to outcome codes
search_substrings = ["walks", "walked", "hit by pitch", "singles", "doubles", "triples", "homers", "strikes"]
mapping = ["walk", "walk", "HBP", "1B", "2B", "3B", "HR", "SO"]

for substring, value in zip(search_substrings, mapping):
    # Check if the substring is present in the column
    mask = at_bat_df['description'].str.contains(substring, case=False)
    # Assign the corresponding value to the 'result' column where the mask is True
    at_bat_df.loc[mask, 'play_outcome'] = value

at_bat_df['play_outcome'] = at_bat_df['play_outcome'].fillna("IPO")

In [7]:
#Mapping outcomes to multi-class targets
outcome_mapping = {"walk": 1,
                   "HBP": 1,
                   "1B": 1,
                   "2B": 2,
                   "3B": 3,
                   "HR": 4,
                   "SO": 0,
                   "IPO": 0}

at_bat_df["mc_target"] = at_bat_df["play_outcome"].map(lambda x: outcome_mapping[x])

In [8]:
#Mapping multi-class targets to binary targets, final target for model
at_bat_df["y_target"] = at_bat_df["mc_target"].map(lambda y: 0 if y  == 0 else 1)

In [14]:
#Setting at_bat id as index
at_bat_df = at_bat_df.set_index("id")

In [15]:
at_bat_df["y_target"].value_counts(normalize=True)

y_target
0    0.681014
1    0.318986
Name: proportion, dtype: float64

In [17]:
#Writing final raw data set with targets to .csv to share and reuse
at_bat_df.to_csv('../raw_data/all_ab_raw_data_w_target.csv', index=True)